In [1]:
import pandas as pd
import numpy as np
import random
import os

from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold, cross_val_score

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

from sklearn.metrics import mean_absolute_percentage_error
import warnings
warnings.filterwarnings("ignore")

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

# 한글 폰트 사용을 위해서 세팅
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/H2GTRM.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

# feature importance 
def vis_feature_importances_(est, X_train):
    ftr_importances_values = est.feature_importances_
    ftr_importances = pd.Series(ftr_importances_values,index=X_train.columns  )
    ftr_top20 = ftr_importances.sort_values(ascending=False)[:20]
    plt.figure(figsize=(10,4))
    plt.title('Feature importances Top 20')
    sns.barplot(x=ftr_top20 , y = ftr_top20.index)
    plt.show()
    
# https://dacon.io/forum/401657    
def smape(true, pred):
    v = 2 * abs(pred - true) / (abs(pred) + abs(true))
    output = np.mean(v) * 100
    return output    

def smape_model():
    def smape(true, pred):
        v = 2 * abs(pred - true) / (abs(pred) + abs(true))
        output = np.mean(v) * 100
        return output    

## Data Load

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

# train = pd.read_csv('/content/drive/MyDrive/work/input/train.csv')
# test = pd.read_csv('/content/drive/MyDrive/work/input/test.csv')
# building_info = pd.read_csv('/content/drive/MyDrive/work/input/building_info.csv')
# submission = pd.read_csv('/content/drive/MyDrive/work/input/sample_submission.csv')
# train.shape, test.shape, building_info.shape, submission.shape

train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')
building_info = pd.read_csv('input/building_info.csv')
submission = pd.read_csv('input/sample_submission.csv')
train.shape, test.shape, building_info.shape, submission.shape

((204000, 10), (16800, 7), (100, 7), (16800, 2))

## Train Data Pre-Processing

In [4]:
#시계열 특성을 학습에 반영하기 위해 일시를 월, 일, 시간으로 나눕니다
train['month'] = train['일시'].apply(lambda x : int(x[4:6]))
train['day'] = train['일시'].apply(lambda x : int(x[6:8]))
train['time'] = train['일시'].apply(lambda x : int(x[9:11]))

test['month'] = test['일시'].apply(lambda x : int(x[4:6]))
test['day'] = test['일시'].apply(lambda x : int(x[6:8]))
test['time'] = test['일시'].apply(lambda x : int(x[9:11]))

# 요일
train['weekday'] =  pd.to_datetime(train['일시'].str[:8]).dt.weekday
test['weekday'] =  pd.to_datetime(test['일시'].str[:8]).dt.weekday

In [5]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

kmeans = pd.DataFrame()

for i in range(1, 101):
#     print(i)
    temp = train[train['건물번호'] == i]
    temp['date'] = pd.to_datetime(temp['일시']).dt.strftime('%Y%m%d')
    temp = temp[['date','weekday','time', '전력소비량(kWh)']]
    temp['kmeans'] = KMeans(n_clusters=2, random_state=42).fit_predict(temp)
    temp['건물번호'] = i
    kmeans = pd.concat([kmeans, temp.reset_index()[['건물번호','weekday','time', 'kmeans']]])
kmeans = kmeans.groupby(['건물번호','weekday','time'])['kmeans'].mean().reset_index()        

kmeans['kmeans_o'] = np.where(kmeans['kmeans'] > 0.6, 1, 0)
train = train.merge(kmeans[['건물번호', 'weekday','time','kmeans_o']], on=['건물번호', 'weekday','time'])
test = test.merge(kmeans[['건물번호', 'weekday','time','kmeans_o']], on=['건물번호', 'weekday','time'])

In [6]:
# 빌딩 데이터 정리
## null 처리
building_info['태양광용량(kW)'] = building_info['태양광용량(kW)'].str.replace('-','0').astype(float)
building_info['ESS저장용량(kWh)'] = building_info['ESS저장용량(kWh)'].str.replace('-','0').astype(float)
building_info['PCS용량(kW)'] = building_info['PCS용량(kW)'].str.replace('-','0').astype(float)

In [7]:
# 정리된 빌딩과 merge
train = train.merge(building_info, on='건물번호')
test = test.merge(building_info, on='건물번호')

In [8]:
# outlier 제거 
train = train.drop(train[train['num_date_time'].isin(['56_20220608 16'
                                                  ,'34_20220808 21'
                                                  ,'10_20220607 06','10_20220607 06'
                                                  ,'31_20220808 13','31_20220809 13', '31_20220808 14','31_20220808 13'
                                                  , '8_20220806 05', '8_20220707 08', '8_20220628 09'
                                                  ,'13_20220606 16','13_20220724 06','13_20220606 17','13_20220607 07'
                                                  ,'58_20220804 16'
                                                  ,'65_20220807 18'
                                                  ,'68_20220706 20','68_20220706 19'
                                                  ,'71_20220604 10','71_20220810 15','71_20220810 16'
                                                  , '73_20220707 12', '73_20220707 14', '73_20220707 11','73_20220707 13', '73_20220707 15', '73_20220808 13','73_20220803 14', '73_20220808 12', '73_20220803 15','73_20220808 11'
                                                 ])].index)

In [9]:
train['강수량(mm)'].fillna(0, inplace=True)
train['풍속(m/s)'].fillna(train['풍속(m/s)'].mean(), inplace=True)
train['습도(%)'].fillna(train['습도(%)'].mean(), inplace=True)

test['강수량(mm)'].fillna(0, inplace=True)
test['풍속(m/s)'].fillna(test['풍속(m/s)'].mean(), inplace=True)
test['습도(%)'].fillna(test['습도(%)'].mean(), inplace=True)

In [10]:
############## 체감온도, https://www.weather.go.kr/plus/life/li_asset/HELP/basic/help_01_07.jsp
train['체감온도'] = 13.12 + 0.6215*train['기온(C)'] - 11.37*(train['풍속(m/s)']*3.6)**0.16 + 0.3965*(train['풍속(m/s)']*3.6)**0.16*train['기온(C)']
test['체감온도'] = 13.12 + 0.6215*test['기온(C)'] - 11.37*(test['풍속(m/s)']*3.6)**0.16 + 0.3965*(test['풍속(m/s)']*3.6)**0.16*test['기온(C)']

In [11]:
no_work_day = train.groupby(['건물번호', '건물유형','month', 'day']).agg({'전력소비량(kWh)':['mean','max']}).reset_index()
no_work_day.columns = ['건물번호', '건물유형','month', 'day', 'khw_mean', 'khw_max']
total_mean = no_work_day[no_work_day['건물유형'].isin(['백화점및아울렛','할인마트'])].groupby(['건물번호']).khw_mean.mean().reset_index().rename(columns={'khw_mean':'b_mean'})

no_work_day = no_work_day.merge(total_mean, on='건물번호')
no_work_day.loc[no_work_day.b_mean > no_work_day.khw_max, '백화점휴무일'] = 1

train = train.merge(no_work_day[['건물번호', '건물유형','month', 'day', '백화점휴무일']], on=['건물번호', '건물유형','month', 'day'], how='left')
train['백화점휴무일'] = train['백화점휴무일'].fillna(0).astype(int)
test['백화점휴무일'] = 0

In [12]:
train.drop( train[(train['건물번호'].isin([1,16])) & (train['month'] == 6)].index, inplace=True)

In [13]:
from feature_engine.creation import CyclicalFeatures
cf = CyclicalFeatures()
col = ['time', '기온(C)']
train = pd.concat([train, cf.fit_transform(train[col]).drop(col, axis=1)], axis=1)
test = pd.concat([test, cf.fit_transform(test[col]).drop(col, axis=1)], axis=1)
train.shape, test.shape

((202530, 27), (16800, 24))

In [14]:
from sklearn.preprocessing import SplineTransformer
spline = SplineTransformer(degree=2, n_knots=3)
col = ['풍속(m/s)','냉방면적(m2)']
train = pd.concat([train.reset_index(drop=True), pd.DataFrame(spline.fit_transform(train[col])).reset_index(drop=True)], axis=1)
test = pd.concat([test.reset_index(drop=True), pd.DataFrame(spline.fit_transform(test[col])).reset_index(drop=True)], axis=1)

train.shape, test.shape

((202530, 35), (16800, 32))

In [15]:
days = [18, 19, 20, 21, 22, 23, 24]

X_train = train[~((train['month'] == 8) & (train['day'].isin(days)))].drop(columns=['num_date_time', '일시', '일조(hr)', '일사(MJ/m2)', '전력소비량(kWh)'])
y_train = train[~((train['month'] == 8) & (train['day'].isin(days)))]['전력소비량(kWh)']

X_test = train[(train['month'] == 8) & (train['day'].isin(days))].drop(columns=['num_date_time', '일시', '일조(hr)', '일사(MJ/m2)', '전력소비량(kWh)'])
y_test = train[(train['month'] == 8) & (train['day'].isin(days))]['전력소비량(kWh)']


print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(185730, 30) (16800, 30) (185730,) (16800,)


In [16]:
#결측값을 0으로 채웁니다
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

## Regression Model Fit

In [17]:
#!pip install optuna
import optuna

In [18]:
def objective(trial):
    params = {
        "random_state": 42,
        "n_estimators": trial.suggest_int("n_estimators", 70, 120),
        "objective": "reg:absoluteerror"
    }
    
    model = XGBRegressor(**params)
    model.set_params(**{'objective':smape_model()})
    model.fit(temp_X_train, temp_y_train)
    y_preds = model.predict(temp_X_test)
    return smape(temp_y_test, y_preds)

In [19]:
%%time 
metrics_type = pd.DataFrame()
for i in train['건물유형'].unique():
    temp_X_train = X_train[X_train['건물유형'] == i].drop(['건물유형'], axis=1)
    temp_y_train = y_train.loc[temp_X_train.index]
    temp_X_test = X_test[X_test['건물유형'] == i].drop(['건물유형'], axis=1)
    temp_y_test = y_test.loc[temp_X_test.index]
    
#     model = XGBRegressor(random_state=42)
#     model.set_params(**{'objective':smape_model()})
#     model.fit(temp_X_train, temp_y_train)
#     y_preds = model.predict(temp_X_test)
    study = optuna.create_study()
    study.optimize(objective, n_trials=10)
    optuna.logging.set_verbosity(optuna.logging.CRITICAL)
    temp = pd.DataFrame([i, study.best_value, study.best_trial.params]).T
    temp.columns = ['건물유형', '유형별_score', '유형별_param']    
#     temp = pd.DataFrame([i, smape(temp_y_test, y_preds)]).T
#     temp.columns = ['건물유형', '유형별_score']
    metrics_type = metrics_type.append(temp)
#     display(temp)
#     vis_feature_importances_(model, temp_X_train)    
    

[I 2023-08-26 20:31:56,082] A new study created in memory with name: no-name-c48f1e0c-fb6f-4aa9-a41e-1fd9ba9b880c
[I 2023-08-26 20:31:59,759] Trial 0 finished with value: 8.525670693166163 and parameters: {'n_estimators': 99}. Best is trial 0 with value: 8.525670693166163.
[I 2023-08-26 20:32:03,159] Trial 1 finished with value: 8.44534823328817 and parameters: {'n_estimators': 117}. Best is trial 1 with value: 8.44534823328817.
[I 2023-08-26 20:32:05,589] Trial 2 finished with value: 8.538652277580432 and parameters: {'n_estimators': 92}. Best is trial 1 with value: 8.44534823328817.
[I 2023-08-26 20:32:08,042] Trial 3 finished with value: 8.564262074447388 and parameters: {'n_estimators': 96}. Best is trial 1 with value: 8.44534823328817.
[I 2023-08-26 20:32:10,103] Trial 4 finished with value: 8.516029135257797 and parameters: {'n_estimators': 73}. Best is trial 1 with value: 8.44534823328817.
[I 2023-08-26 20:32:12,932] Trial 5 finished with value: 8.503156505930233 and parameters:

Wall time: 3min 41s


In [20]:
%%time 
metrics_bno = pd.DataFrame()
for i in train['건물번호'].unique():

    temp_X_train = X_train[X_train['건물번호'] == i].drop(['건물번호','건물유형'], axis=1)
    temp_y_train = y_train.loc[temp_X_train.index]
    temp_X_test = X_test[X_test['건물번호'] == i].drop(['건물번호','건물유형'], axis=1)
    temp_y_test = y_test.loc[temp_X_test.index]
        
#     model = XGBRegressor(random_state=42)
#     model.set_params(**{'objective':smape_model()})
#     model.fit(temp_X_train, temp_y_train)
#     y_preds = model.predict(temp_X_test)

    study = optuna.create_study()
    study.optimize(objective, n_trials=10)
    optuna.logging.set_verbosity(optuna.logging.CRITICAL)
    temp = pd.DataFrame([i, study.best_value, study.best_trial.params]).T
    temp.columns = ['건물번호', '건물별_score', '건물별_param']
    
    metrics_bno = metrics_bno.append(temp)
#     display(temp)
#     vis_feature_importances_(model, temp_X_train)    
#     print()

Wall time: 8min 33s


In [21]:
metrics_bno['건물번호'] = metrics_bno['건물번호'].astype(int)

In [22]:
metrics_bno = metrics_bno.merge(building_info[['건물유형','건물번호']], on=['건물번호'])

In [23]:
metrics = metrics_bno.merge(metrics_type, on=['건물유형'])
metrics[:2]

,건물번호,건물별_score,건물별_param,건물유형,유형별_score,유형별_param
0,1,6.186132,{'n_estimators': 72},건물기타,8.445348,{'n_estimators': 117}
1,2,7.440776,{'n_estimators': 98},건물기타,8.445348,{'n_estimators': 117}


In [24]:
# 유형보다 건물이 더 좋은 스코어를 가지는 건물번호 list
bno = metrics[metrics['건물별_score'] < metrics['유형별_score']]['건물번호'].values
len(bno), bno

(72,
 array([  1,   2,   4,   5,   6,   7,   8,   9,  11,  12,  15,  16,  17,
         18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  32,  33,  35,
         37,  38,  39,  41,  42,  44,  45,  48,  49,  50,  52,  55,  56,
         57,  58,  59,  60,  61,  62,  63,  64,  66,  68,  69,  70,  71,
         73,  76,  77,  78,  79,  80,  81,  83,  84,  85,  86,  88,  89,
         92,  93,  94,  96,  97,  99, 100]))

In [25]:
metrics_type['유형별_score'].mean(), metrics_bno['건물별_score'].mean()
# (7.026821963284757, 6.107093822048833)

(6.0415841225848155, 5.599964981989843)

In [26]:
# 가중 평균 
metrics_type['유형별_score'].mean() * ((100-len(bno))/100) + metrics_bno['건물별_score'].mean() * (len(bno)/100)

5.723618341356436

In [ ]:
# 5.782455883802
# 5.77847481671447 0.6

In [ ]:
metrics['score'] = metrics['유형별_score'] 
metrics.loc[metrics['건물번호'].isin(bno), 'score'] = metrics['건물별_score']

In [ ]:
metrics['건물이낫다'] = metrics['건물별_score'] < metrics['유형별_score']

In [ ]:
# metrics.to_excel('오차큰것부터_20230810-3.xlsx')

## Inference & Submit

In [25]:
%%time
result_type = pd.DataFrame()
for i in train['건물유형'].unique():
    
    temp_X_train = train[train['건물유형'] == i].drop(['건물유형','num_date_time', '일시', '일조(hr)', '일사(MJ/m2)', '전력소비량(kWh)'], axis=1)
    temp_y_train = train.loc[temp_X_train.index][['전력소비량(kWh)']]
    
    model = XGBRegressor(n_estimators=metrics_type[metrics_type['건물유형'] == i]['유형별_param'][0]['n_estimators'],random_state=42)
    model.set_params(**{'objective':smape_model()})
    
    model.fit(temp_X_train, temp_y_train)
                                                 
    temp_test = test[test['건물유형'] == i].drop(['건물유형','num_date_time', '일시'], axis=1)
    y_preds = model.predict(temp_test)
    result_type = result_type.append( pd.concat([test[test['건물유형'] == i].reset_index(), pd.DataFrame(y_preds, columns=['answer_유형별']).reset_index()], axis=1) )
    
result_type.shape    

Wall time: 11.6 s


(16800, 35)

In [26]:
%%time
result_bno = pd.DataFrame()
for i in train['건물번호'].unique():
    
    temp_X_train = train[train['건물번호'] == i].drop(['건물번호','건물유형','num_date_time', '일시', '일조(hr)', '일사(MJ/m2)', '전력소비량(kWh)'], axis=1)
    temp_y_train = train.loc[temp_X_train.index][['전력소비량(kWh)']]
    
    model = XGBRegressor(n_estimators=metrics_bno[metrics_bno['건물번호'] == i]['건물별_param'].values[0]['n_estimators'], random_state=42)
    model.set_params(**{'objective':smape_model()})
    
    model.fit(temp_X_train, temp_y_train)
                                                 
    temp_test = test[test['건물번호'] == i].drop(['건물번호','건물유형','num_date_time', '일시'], axis=1)
    y_preds = model.predict(temp_test)
    result_bno = result_bno.append( pd.concat([test[test['건물번호'] == i].reset_index(), pd.DataFrame(y_preds, columns=['answer_건물별']).reset_index()], axis=1) )
    
result_bno.shape    

Wall time: 23.4 s


(16800, 35)

In [27]:
result = result_type[['num_date_time','건물번호','answer_유형별','weekday', 'time']].merge(result_bno[['num_date_time','건물번호','answer_건물별']], on=['num_date_time','건물번호'])
result['answer'] = result['answer_유형별'] 
result.loc[result['건물번호'].isin(bno), 'answer'] = result['answer_건물별']

# 후처리 
postprocessing = train.groupby(['건물번호', 'weekday','time'])['전력소비량(kWh)'].min().reset_index()
result = result.merge(postprocessing, on=['건물번호', 'weekday','time'])
result.loc[result['answer'] < result['전력소비량(kWh)'], 'answer'] = result['전력소비량(kWh)']

result[['num_date_time','answer']].to_csv('20230826-1_x.csv', index=False)